In [1]:
import alpaca_trade_api as tradeapi
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
API_KEY = os.getenv('API_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')
END_POINT = os.getenv('END_POINT')


In [3]:
api = tradeapi.REST(API_KEY, SECRET_KEY, END_POINT, api_version='v2')

In [ ]:
symbol = 'BYND'
timeframe = '1min'  
start_date = '2021-01-01'
end_date = '2022-01-01'

bars = api.get_bars(symbol, timeframe, start=start_date, end=end_date).df

bars.index = pd.to_datetime(bars.index)
bars = bars[['open', 'high', 'low', 'close', 'volume']]
# Calculate EMA 
bars['ema_12'] = ta.ema(bars['close'], length=12)
bars['ema_26'] = ta.ema(bars['close'], length=26)

bars.dropna(inplace=True)


bars['signal'] = 0
bars['signal'][bars['ema_12'] < bars['ema_26']] = 1  # Buy
bars['signal'][bars['ema_12'] > bars['ema_26']] = -1  # Sell

bars['position'] = 0  
bars

In [39]:

positions = [0]  

for i in range(1, len(bars)):
    if bars['signal'].iloc[i] == 1 and positions[-1] == 0:
        positions.append(1)  # Buy
    elif bars['signal'].iloc[i] == -1 and positions[-1] == 1:
        positions.append(0)  # Sell
    else:
        positions.append(positions[-1]) # Hold

bars['position'] = positions

In [ ]:
portfolio_values = []
stock_values = []

cash = 10000 
position = 0  
shares = 0  

# Simulate trading
for index, row in bars.iterrows():
    # Buy 
    if row['position'] == 1 and cash >= row['close']: 
        shares_to_buy = cash // row['close'] 
        buy_price = row['close']
        cash -= shares_to_buy * buy_price  
        shares += shares_to_buy  
        print(f"Buying {shares_to_buy} shares @ {buy_price} on {index}")

    # Sell 
    elif row['position'] == 0 and shares > 0:
        sell_price = row['close']
        cash += shares * sell_price 
        shares = 0  
        print(f"Selling all shares @ {sell_price} on {index}")

    portfolio_value = cash + shares * row['close']
    portfolio_values.append(portfolio_value)

    stock_values.append(row['close'])

portfolio_value = cash + shares * bars.iloc[-1]['close']
print(f"Final Portfolio Value: ${portfolio_value:.2f}")

In [ ]:
performance_df = pd.DataFrame({
    'portfolio_value': portfolio_values,
    'stock_value': stock_values
}, index=bars.index)


performance_df['portfolio_return'] = (performance_df['portfolio_value'] / performance_df['portfolio_value'].iloc[0] - 1) * 100
performance_df['stock_return'] = (performance_df['stock_value'] / performance_df['stock_value'].iloc[0] - 1) * 100

plt.figure(figsize=(12, 6))

plt.plot(performance_df.index, performance_df['stock_return'], label=f'{symbol} Stock Return (%)', color='blue')

plt.plot(performance_df.index, performance_df['portfolio_return'], label='Portfolio Return (%)', color='green')

plt.title(f'{symbol} Stock Return (%) vs. Portfolio Return (%) (EMA Crossover Strategy)')
plt.xlabel('Date')
plt.ylabel('Return (%)')
plt.legend()

plt.show()